In [6]:
import MDAnalysis as mda
import numpy as np
import pandas as pd
import sys
sys.path.append('/Users/dburns/Library/CloudStorage/Box-Box/my_scripts')
from ChACRA.ContactAnalysis.ContactFrequencies import *

In [2]:
u = mda.Universe('/Users/dburns/Library/CloudStorage/Box-Box/1_GROUP_data/Enzymes/atcase/from_pronto/1q95/1q95_minimized_nohoh.pdb')

In [13]:
protein = u.select_atoms('protein')

In [18]:
test = mda.Merge(protein)
test 

<Universe with 43566 atoms>

In [23]:
universe = test
residues = {seg.segid: seg.residues.resnames for seg in universe.segments}
segids = list(residues.keys())
# make a square matrix that will be filled with True values for identical subunits
array = np.zeros((len(segids),len(segids)),dtype=np.bool_)
# every subunit is identical with itself
np.fill_diagonal(array,True)
# work with it as a df
identical_table = pd.DataFrame(array, columns=segids, index=segids)



In [24]:
identical_table

,A,B,C,D,E,F,G,H,I,J,K,L
A,True,False,False,False,False,False,False,False,False,False,False,False
B,False,True,False,False,False,False,False,False,False,False,False,False
C,False,False,True,False,False,False,False,False,False,False,False,False
D,False,False,False,True,False,False,False,False,False,False,False,False
E,False,False,False,False,True,False,False,False,False,False,False,False
F,False,False,False,False,False,True,False,False,False,False,False,False
G,False,False,False,False,False,False,True,False,False,False,False,False
H,False,False,False,False,False,False,False,True,False,False,False,False
I,False,False,False,False,False,False,False,False,True,False,False,False
J,False,False,False,False,False,False,False,False,False,True,False,False


In [26]:
combo[0]

'A'

In [27]:
# Go through all pairwise combinations of subunits
for combo in combinations(segids,2):
    # not identical if lengths are different
    if len(residues[combo[0]]) != len(residues[combo[1]]):
         identical_table[combo[1]][combo[0]], identical_table[combo[0]][combo[1]] = False, False
    else:
        bool = np.all(np.equal(residues[combo[0]],residues[combo[1]]))
        # Enter True or False in both the upper tri and lower tri  
        identical_table[combo[1]][combo[0]], identical_table[combo[0]][combo[1]] =  bool, bool

# Only keep one representative row that describes the identical sets of subunits
identical_table.drop_duplicates(inplace=True)

identical_subunits = {}
# Add the lists of identical subunits to the dictionary
# You can deal with different sets of identical subunits in complex situations\
for i, segid in enumerate(identical_table.index):
    subunits = identical_table.T[identical_table.loc[segid]==True].index
    # if there is only one, it's just identical with itself
    if len(subunits) >= 2:
        identical_subunits[i] = list(subunits)

In [28]:
identical_table

,A,B,C,D,E,F,G,H,I,J,K,L
A,True,True,True,True,True,True,False,False,False,False,False,False
G,False,False,False,False,False,False,True,True,True,True,True,True


In [5]:
segids = set(u.residues.segids)
# hold the coms for each protomer 
seg_sels = {seg: u.select_atoms(f'segid {seg}') for seg in set(u.residues.segids)} # if segid in subunits

In [6]:
# access a subunit com
from itertools import combinations, permutations
triples = [[a,b,c] for a,b,c in permutations(segids, 3)]
coms = {segid: seg_sels[segid].center_of_mass() for segid in segids}


In [7]:
import numpy as np
# triple selections without duplicating angle created by vertex and permutations of the 2 points
trips = []
for segid in segids:
    trips.append([[ch1,segid,ch2] for ch1,ch2 in combinations([ch for ch in segids if ch != segid],2)])

In [8]:
trips

[[['A', 'C', 'D'], ['A', 'C', 'B'], ['D', 'C', 'B']],
 [['C', 'A', 'D'], ['C', 'A', 'B'], ['D', 'A', 'B']],
 [['C', 'D', 'A'], ['C', 'D', 'B'], ['A', 'D', 'B']],
 [['C', 'B', 'A'], ['C', 'B', 'D'], ['A', 'B', 'D']]]

In [9]:
all_seg_sel_string = ''.join([f'segid {seg} or ' for seg in list(segids)[:-1]])
all_seg_sel_string += f'segid {list(segids)[-1]}'
all_com = u.select_atoms(all_seg_sel_string).center_of_mass()

In [10]:
def get_angle(a,b,c):
    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return angle

In [11]:
opposing_subunits = []
check = []
for seg in segids:
    if seg not in check:
        for seg2 in [segid for segid in segids if segid != seg]:
            if np.abs(180-np.rad2deg(get_angle(coms[seg],all_com,coms[seg2]))) <= 2:
                opposing_subunits.append((seg,seg2))
                check.append(seg)
                break




In [12]:
opposing_subunits

[('C', 'B'), ('A', 'D'), ('D', 'A'), ('B', 'C')]

In [80]:
np.rad2deg(get_angle(coms['A'],all_com,coms['D']))

179.92590933219583